# ChalkJuice NFL MySQl Project

# input the gold_all_49.csv into the games MySQL database

### This code converts data stored in .csv files to a MySQL tables

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import getpass
#!pip install pymysql
import pymysql as mysql
import os
import seaborn as sns
import matplotlib.pyplot as plt

#!pip install tbats
from tbats import TBATS
import pandas as pd
import matplotlib.pyplot as plt

## connect to MySQL

In [2]:
cnx = mysql.connect(
        host='localhost',
        user='root',
        passwd=os.getenv('MYSQL'),
        database='nfl',
        port=int(3306)
)

In [3]:
cur = cnx.cursor()

In [4]:
def sql(query, params=None):
    cur = cnx.cursor()
    cur.execute(query)
    for x in cur:
        print(x)

In [5]:
def todf(query):
    df = pd.read_sql(query, cnx)
    return df

In [6]:
a = sql('SHOW TABLES;')

('games',)


In [7]:
df = pd.read_csv('gold_all_49.csv')
#df = df[df['game_year'] > 1998]
df.replace('', np.nan, inplace=True)
df[df.select_dtypes(include=['float64']).columns] = df.select_dtypes(include=['float64']).astype('Int64')
df['game_date'] = df['game_date'].astype('datetime64[ns]')

In [8]:
df.insert(0, 'game_id', range(1, len(df) + 1))
df.head(3)

,game_id,game_date,game_year,game_team,game_opp,game_result,game_team_pts,game_opp_pts,game_overtime,game_ending_tds,...,speams_XPR,defense_tds_int,defense_tds_fmb,speams_tds_KR,speams_tds_PR,speams_tds_blocked_fg,speams_tds_blocked_punt,game_tds_oth,downs_1D_pass,downs_1D_run
0,1,2023-01-02,2022,BUF,CIN,C,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2023-01-02,2022,CIN,BUF,C,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1999-09-12,1999,CLE,PIT,L,0,43,0,0,...,0,0,0,0,0,0,0,0,2,0


## check for completeness of dataset by year and ensure no overlooked null values

In [9]:
nan_counts = df.groupby('game_year').apply(lambda x: x.isna().sum())
nan_counts = nan_counts.loc[:, (nan_counts != 0).any(axis=0)]
nan_counts.head(25)


C:\Users\joel\AppData\Local\Temp\ipykernel_23776\1129923490.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  nan_counts = df.groupby('game_year').apply(lambda x: x.isna().sum())


,game_time_total,downs_3D_att,downs_3D_con,downs_4D_att,downs_4D_con,game_time_off,downs_1D_pass,downs_1D_run
game_year,,,,,,,,
1967,350,350,350,350,350,350,350,350
1968,364,364,364,364,364,364,364,364
1969,364,364,364,364,364,364,364,364
1970,364,364,364,364,364,364,364,364
1971,364,364,364,364,364,364,364,364
1972,364,364,364,364,364,364,364,364
1973,364,364,364,364,364,364,364,364
1974,364,364,364,364,364,364,364,364
1975,364,364,364,364,364,364,364,364


## optional: create template for table

In [10]:
for col in df.columns[:]:
    x = df.dtypes[col]
    if x == 'int64':
        x = 'INT'
    elif x == 'Int64':
        x = 'INT'
    elif x == 'float64':
        x = 'FLOAT'
    elif x == 'object':
        x = 'VARCHAR(255)'
    elif x == 'datetime64[ns]':
        x = 'DATETIME'
    else:
        x = 'VARCHAR(255)'
    y = col
    print(col, x, ',')


game_id INT ,
game_date DATETIME ,
game_year INT ,
game_team VARCHAR(255) ,
game_opp VARCHAR(255) ,
game_result VARCHAR(255) ,
game_team_pts INT ,
game_opp_pts INT ,
game_overtime INT ,
game_ending_tds INT ,
game_time_total INT ,
game_home INT ,
passing_com INT ,
passing_att INT ,
passing_yds INT ,
passing_tds INT ,
passing_int INT ,
passing_sacks_total INT ,
passing_sacks_yards INT ,
rushing_att INT ,
rushing_yds INT ,
rushing_tds INT ,
rushing_fmb INT ,
downs_3D_att INT ,
downs_3D_con INT ,
downs_4D_att INT ,
downs_4D_con INT ,
game_time_off INT ,
speams_XPM INT ,
speams_XPA INT ,
speams_FGM INT ,
speams_FGA INT ,
game_pen_total INT ,
game_pen_yds INT ,
speams_punts_total INT ,
speams_punts_yds INT ,
speams_punts_blocks INT ,
speams_2PM INT ,
speams_2PA INT ,
defense_saf INT ,
speams_XPR INT ,
defense_tds_int INT ,
defense_tds_fmb INT ,
speams_tds_KR INT ,
speams_tds_PR INT ,
speams_tds_blocked_fg INT ,
speams_tds_blocked_punt INT ,
game_tds_oth INT ,
downs_1D_pass INT ,
downs_1D_run

## generate empty MySQL table

In [11]:
# create query
create_table = """
    CREATE TABLE IF NOT EXISTS games(
        game_id INT PRIMARY KEY,
        game_date DATE NOT NULL,
        game_year INT,
        game_team VARCHAR(255) NOT NULL,
        game_opp VARCHAR(255) NOT NULL,
        game_result VARCHAR(255),
        game_team_pts INT ,
        game_opp_pts INT ,
        game_overtime INT ,
        game_ending_tds INT ,
        game_time_total INT ,
        game_home INT ,
        passing_com INT ,
        passing_att INT ,
        passing_yds INT ,
        passing_tds INT ,
        passing_int INT ,
        passing_sacks_total INT ,
        passing_sacks_yards INT ,
        rushing_att INT ,
        rushing_yds INT ,
        rushing_tds INT ,
        rushing_fmb INT ,
        downs_3D_att INT ,
        downs_3D_con INT ,
        downs_4D_att INT ,
        downs_4D_con INT ,
        game_time_off INT ,
        speams_XPM INT ,
        speams_XPA INT ,
        speams_FGM INT ,
        speams_FGA INT ,
        game_pen_total INT ,
        game_pen_yds INT ,
        speams_punts_total INT ,
        speams_punts_yds INT ,
        speams_punts_blocks INT ,
        speams_2PM INT ,
        speams_2PA INT ,
        defense_saf INT ,
        speams_XPR INT ,
        defense_tds_int INT ,
        defense_tds_fmb INT ,
        speams_tds_KR INT ,
        speams_tds_PR INT ,
        speams_tds_blocked_fg INT ,
        speams_tds_blocked_punt INT ,
        game_tds_oth INT ,
        downs_1D_pass INT ,
        downs_1D_run INT
    )
;
"""

# execute query
sql(create_table)

## change all pandas 'Nan' to string 'None' for MySQL

In [12]:
df = df.replace({pd.NA: None}).where(pd.notnull(df), None)

In [13]:
df.head(10)

,game_id,game_date,game_year,game_team,game_opp,game_result,game_team_pts,game_opp_pts,game_overtime,game_ending_tds,...,speams_XPR,defense_tds_int,defense_tds_fmb,speams_tds_KR,speams_tds_PR,speams_tds_blocked_fg,speams_tds_blocked_punt,game_tds_oth,downs_1D_pass,downs_1D_run
0,1,2023-01-02,2022,BUF,CIN,C,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2023-01-02,2022,CIN,BUF,C,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1999-09-12,1999,CLE,PIT,L,0,43,0,0,...,0,0,0,0,0,0,0,0,2,0
3,4,1983-11-20,1983,PHI,NYG,L,0,23,0,0,...,0,0,0,0,0,0,0,0,None,None
4,5,2009-09-21,2009,IND,MIA,W,27,23,0,0,...,0,0,0,0,0,0,0,0,11,3
5,6,2008-11-02,2008,LVR,ATL,L,0,24,0,0,...,0,0,0,0,0,0,0,0,0,3
6,7,1999-10-17,1999,GNB,DEN,L,10,31,0,0,...,0,0,0,0,0,0,0,0,4,1
7,8,1991-10-20,1991,DET,SFO,L,3,35,0,0,...,0,0,0,0,0,0,0,0,None,None
8,9,1988-09-04,1988,MIA,CHI,L,7,34,0,0,...,0,0,0,0,0,0,0,0,None,None
9,10,2011-11-20,2011,ARI,SFO,L,7,23,0,0,...,0,0,0,0,0,0,0,0,7,1


## add pandas df to MySQL table

### this is hidden behind # because it has already successfully been added

In [17]:
# list column names to match columns in table
#cols = ', '.join([f'`{col}`' for col in df.columns])

# create a placeholder for each column
#placeholders = ', '.join(['%s'] * len(df.columns))

# generate each individual query (by df row)
#query = f"INSERT INTO games ({cols}) VALUES ({placeholders})"

# convert df to list of tuples
#data = [tuple(row) for row in df.to_numpy()]

# execute queries
#cur.executemany(query, data)


In [18]:
cnx.commit()